# Concurrent Orchestration

The concurrent pattern enables multiple agents to work on the same task in parallel. Each agent processes the input independently, and their results are collected and aggregated. This approach is well-suited for scenarios where diverse perspectives or solutions are valuable, such as brainstorming, ensemble reasoning, or voting systems.

* [Agent-orchestration - Concurrent](https://learn.microsoft.com/en-us/semantic-kernel/frameworks/agent/agent-orchestration/concurrent?pivots=programming-language-python)

The following sample demonstrates how to create a concurrent orchestration for executing multiple agents on the same task in parallel.

This sample demonstrates the basic steps of creating and starting a runtime, creating a concurrent orchestration with multiple agents, invoking the orchestration, and finally waiting for the results.

In [1]:
#Libraries

from dotenv import load_dotenv
import os
import asyncio

from semantic_kernel.agents import Agent, ChatCompletionAgent, ConcurrentOrchestration
from semantic_kernel.agents.runtime import InProcessRuntime
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

In [2]:
#load variables
load_dotenv()

# Variables - Azure Services
azopenai_ep=os.environ["AZURE_OPENAI_ACCOUNT"]
azopenai_key=os.environ["AZURE_OPENAI_KEY"]
azopenai_model=os.environ["AZURE_OPENAI_MODEL"]
azopenai_api_ver=os.environ["AZURE_OPENAI_API_VERSION"]

### Create AzureChatCommpletion Class

In [3]:
chat_physics_agent = AzureChatCompletion(
    deployment_name=azopenai_model,
    api_key=azopenai_key,
    endpoint=azopenai_ep,
    service_id="physics_agent"
)

chat_chemistry_agent = AzureChatCompletion(
    deployment_name=azopenai_model,
    api_key=azopenai_key,
    endpoint=azopenai_ep,
    service_id="chemistry_agent"
)

### Create the agents

In [4]:
def get_agents() -> list[Agent]:
    """Return a list of agents that will participate in the concurrent orchestration.

    Feel free to add or remove agents.
    """
    physics_agent = ChatCompletionAgent(
        name="PhysicsExpert",
        instructions="You are an expert in physics. You answer questions from a physics perspective.",
        service=chat_physics_agent
    )
    chemistry_agent = ChatCompletionAgent(
        name="ChemistryExpert",
        instructions="You are an expert in chemistry. You answer questions from a chemistry perspective.",
        service=chat_chemistry_agent
    )

    return [physics_agent, chemistry_agent]

### Create main logic

In [8]:
# 1. Create a concurrent orchestration with multiple agents
agents = get_agents()
concurrent_orchestration = ConcurrentOrchestration(members=agents)

# 2. Create a runtime and start it
runtime = InProcessRuntime()
runtime.start()

# 3. Invoke the orchestration with a task and the runtime
orchestration_result = await concurrent_orchestration.invoke(
    task="What is temperature?",
    runtime=runtime
)

In [ ]:
# 4. Wait for the results
# Note: the order of the results is not guaranteed to be the same
# as the order of the agents in the orchestration.
value = await orchestration_result.get(timeout=20)

for item in value:
    print(f"# {item.name}: {item.content}\n\n***********************\n")

# 5. Stop the runtime after the invocation is complete
await runtime.stop_when_idle()

# ChemistryExpert: In chemistry, *temperature* is a fundamental physical property that measures the average kinetic energy of the particles—atoms, molecules, or ions—in a system. It essentially indicates how hot or cold a substance is and reflects the energy associated with the random motion of its particles.

### Key points:
1. **Kinetic Energy and Particle Motion**:
   - At the microscopic level, particles in any material are in constant motion (vibrational, rotational, or translational, depending on the state of matter). The temperature of a substance is directly proportional to the average kinetic energy of these particles.
   - Higher temperature means particles are moving faster, while lower temperature corresponds to slower motion.

2. **Temperature Scales**:
   - **Kelvin (K)**: The absolute temperature scale often used in scientific contexts, especially in chemistry, because it starts at absolute zero (theoretically the point where all molecular motion ceases).
   - **Celsius 